In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2018 data into respective dataframes
df1 = pd.read_csv('../Data/2018/Divvy_Trips_2018_Q1.csv')
df2 = pd.read_csv('../Data/2018/Divvy_Trips_2018_Q2.csv')
df3 = pd.read_csv('../Data/2018/Divvy_Trips_2018_Q3.csv')
df4 = pd.read_csv('../Data/2018/Divvy_Trips_2018_Q4.csv')

In [3]:
df1.columns

Index(['01 - Rental Details Rental ID', '01 - Rental Details Local Start Time',
       '01 - Rental Details Local End Time', '01 - Rental Details Bike ID',
       '01 - Rental Details Duration In Seconds Uncapped',
       '03 - Rental Start Station ID', '03 - Rental Start Station Name',
       '02 - Rental End Station ID', '02 - Rental End Station Name',
       'User Type', 'Member Gender',
       '05 - Member Details Member Birthday Year'],
      dtype='object')

In [4]:
df2.columns

Index(['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

### The df1 has column names which are different than the others (df2, df3, df4 have similar column names). So renaming the columns as required

In [5]:
df1 = df1.rename(columns={"User Type":"usertype",
                          "03 - Rental Start Station Name":"from_station_name",
                          "02 - Rental End Station Name": "to_station_name",
                         "01 - Rental Details Local Start Time": "start_time",
                         "01 - Rental Details Local End Time": "end_time"})

In [6]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,01 - Rental Details Rental ID,start_time,end_time,01 - Rental Details Bike ID,01 - Rental Details Duration In Seconds Uncapped,03 - Rental Start Station ID,from_station_name,02 - Rental End Station ID,to_station_name,usertype,Member Gender,05 - Member Details Member Birthday Year,trip_id,bikeid,tripduration,from_station_id,to_station_id,gender,birthyear
0,17536702.0,2018-01-01 00:12:00,2018-01-01 00:17:23,3304.0,323.0,69.0,Damen Ave & Pierce Ave,159.0,Claremont Ave & Hirsch St,Subscriber,Male,1988.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17536703.0,2018-01-01 00:41:35,2018-01-01 00:47:52,5367.0,377.0,253.0,Winthrop Ave & Lawrence Ave,325.0,Clark St & Winnemac Ave (Temp),Subscriber,Male,1984.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17536704.0,2018-01-01 00:44:46,2018-01-01 01:33:10,4599.0,"2,904.0",98.0,LaSalle St & Washington St,509.0,Troy St & North Ave,Subscriber,Male,1989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17536705.0,2018-01-01 00:53:10,2018-01-01 01:05:37,2302.0,747.0,125.0,Rush St & Hubbard St,364.0,Larrabee St & Oak St,Subscriber,Male,1983.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17536706.0,2018-01-01 00:53:37,2018-01-01 00:56:40,3696.0,183.0,129.0,Blue Island Ave & 18th St,205.0,Paulina St & 18th St,Subscriber,Male,1989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dfbike.columns

Index(['01 - Rental Details Rental ID', 'start_time', 'end_time',
       '01 - Rental Details Bike ID',
       '01 - Rental Details Duration In Seconds Uncapped',
       '03 - Rental Start Station ID', 'from_station_name',
       '02 - Rental End Station ID', 'to_station_name', 'usertype',
       'Member Gender', '05 - Member Details Member Birthday Year', 'trip_id',
       'bikeid', 'tripduration', 'from_station_id', 'to_station_id', 'gender',
       'birthyear'],
      dtype='object')

### Saving all the station names in a list "allSt18"

#### Comparing with 2017 data and only adding the stations which are new into a list "added18"
#### Also, checking for the stations that have been in 2017 but not in this year's data and adding it to "removed17"

In [8]:
allSt18 = list(dfbike['from_station_name'].unique())
len(allSt18)

622

In [9]:
allSt17 = ['Clinton St & Lake St',
 'Wabash Ave & Grand Ave',
 'Greenview Ave & Jarvis Ave',
 'Dearborn St & Erie St',
 'Sheffield Ave & Webster Ave',
 'Sedgwick St & Webster Ave',
 'Daley Center Plaza',
 'Desplaines St & Kinzie St',
 'Ashland Ave & Division St',
 'Wilton Ave & Belmont Ave',
 'Southport Ave & Waveland Ave',
 'Wabash Ave & Roosevelt Rd',
 'Sheffield Ave & Fullerton Ave',
 'Ashland Ave & Blackhawk St',
 'Clinton St & Madison St',
 'Millennium Park',
 'Clark St & Lake St',
 'Clinton St & Washington Blvd',
 'Damen Ave & Pierce Ave',
 'McClurg Ct & Illinois St',
 'Clinton St & Jackson Blvd',
 'Lincoln Ave & Roscoe St',
 'Wells St & Concord Ln',
 'Wells St & Evergreen Ave',
 'Green St & Randolph St',
 'Ashland Ave & Harrison St',
 'Wells St & Huron St',
 'Halsted St & Willow St',
 'Clark St & Lincoln Ave',
 'Broadway & Sheridan Rd',
 'Rush St & Cedar St',
 'Mies van der Rohe Way & Chestnut St',
 'Western Ave & Walton St',
 'Broadway & Argyle St',
 'Artesian Ave & Hubbard St',
 'Michigan Ave & Congress Pkwy',
 'Sedgwick St & North Ave',
 'Michigan Ave & Washington St',
 'California Ave & Milwaukee Ave',
 'Clark St & Columbia Ave',
 'Broadway & Cornelia Ave',
 'Aberdeen St & Jackson Blvd',
 'Campbell Ave & Fullerton Ave',
 'Drake Ave & Montrose Ave',
 'Clark St & North Ave',
 'Clark St & Berwyn Ave',
 'Damen Ave & Cortland St',
 'Sheridan Rd & Irving Park Rd',
 'Buckingham Fountain',
 'McClurg Ct & Erie St',
 'Broadway & Barry Ave',
 'Columbus Dr & Randolph St',
 'Leavitt St & North Ave',
 'Wells St & Walton St',
 'Pine Grove Ave & Irving Park Rd',
 'Sedgwick St & Huron St',
 'Eckhart Park',
 'State St & Harrison St',
 'Wilton Ave & Diversey Pkwy',
 'Damen Ave & Division St',
 'Clark St & Wrightwood Ave',
 'Halsted St & Roscoe St',
 'Kenton Ave & Madison St',
 'Paulina Ave & North Ave',
 'Clark St & Wellington Ave',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Green St & Madison St',
 'Michigan Ave & Jackson Blvd',
 'Morgan St & Lake St',
 'Calumet Ave & 33rd St',
 'Dearborn St & Adams St',
 'Glenwood Ave & Morse Ave',
 'Kingsbury St & Erie St',
 'Lakeview Ave & Fullerton Pkwy',
 'Orleans St & Merchandise Mart Plaza',
 'State St & Pearson St',
 'Rush St & Superior St',
 'Clark St & Elm St',
 'Western Ave & Granville Ave',
 'Damen Ave & Madison St',
 'State St & Kinzie St',
 'Franklin St & Chicago Ave',
 'McCormick Place',
 'Sheridan Rd & Montrose Ave',
 'State St & 33rd St',
 'Racine Ave & Belmont Ave',
 'LaSalle St & Illinois St',
 'Canal St & Madison St',
 'Clark St & Chicago Ave',
 'Winchester Ave & Elston Ave',
 'Harper Ave & 59th St',
 'Halsted St & Dickens Ave',
 'Chicago Ave & Sheridan Rd',
 'Logan Blvd & Elston Ave',
 'Kedzie Ave & Palmer Ct',
 'California Ave & Division St',
 'Halsted St & Blackhawk St (*)',
 'Rush St & Hubbard St',
 'Streeter Dr & Grand Ave',
 'Ellis Ave & 55th St',
 'Milwaukee Ave & Wabansia Ave',
 'Richmond St & Diversey Ave',
 'Michigan Ave & Lake St',
 'Wabash Ave & 8th St',
 'Sheffield Ave & Waveland Ave',
 'Michigan Ave & Oak St',
 'Dearborn St & Monroe St',
 'Clark St & Randolph St',
 'Ada St & Washington Blvd',
 'Broadway & Waveland Ave',
 'Christiana Ave & Lawrence Ave',
 'Federal St & Polk St',
 'State St & Randolph St',
 'Campbell Ave & Montrose Ave',
 'Racine Ave & 18th St',
 'Sheffield Ave & Wellington Ave',
 'Pine Grove Ave & Waveland Ave',
 'University Library (NU)',
 'Ellis Ave & 53rd St',
 'University Ave & 57th St',
 'State St & Van Buren St',
 'MLK Jr Dr & 29th St',
 'Sheffield Ave & Wrightwood Ave',
 'Carpenter St & Huron St',
 'Sedgwick St & Schiller St',
 'Orleans St & Ohio St',
 'Damen Ave & Leland Ave',
 'Lake Shore Dr & Belmont Ave',
 'Southport Ave & Roscoe St',
 'May St & Taylor St',
 'Paulina St & Howard St',
 'Larrabee St & Kingsbury St',
 'Financial Pl & Congress Pkwy',
 'Ashland Ave & Wellington Ave',
 'Halsted St & Polk St',
 'Clark St & Congress Pkwy',
 'Wabash Ave & Wacker Pl',
 'Sheffield Ave & Willow St',
 'Lincoln Ave & Fullerton Ave',
 'Peoria St & Jackson Blvd',
 'Calumet Ave & 51st St',
 'Southport Ave & Wrightwood Ave',
 'Mies van der Rohe Way & Chicago Ave',
 'Rhodes Ave & 32nd St',
 'Ellis Ave & 58th St',
 'California Ave & North Ave',
 'Leavitt St & Lawrence Ave',
 'Wood St & Division St',
 'Greenview Ave & Fullerton Ave',
 'Blue Island Ave & 18th St',
 'Larrabee St & Webster Ave',
 'Loomis St & Lexington St',
 'Kingsbury St & Kinzie St',
 'Indiana Ave & 40th St',
 'Clark St & Winnemac Ave',
 'Lawndale Ave & 23rd St',
 'Clifton Ave & Armitage Ave',
 'Stockton Dr & Wrightwood Ave',
 'Lake Shore Dr & Monroe St',
 'Greenview Ave & Diversey Pkwy',
 'Clark St & Montrose Ave',
 'Broadway & Ridge Ave',
 'State St & 79th St',
 'Southport Ave & Clark St',
 'Broadway & Berwyn Ave',
 'Lincoln Ave & Belmont Ave',
 'Canal St & Adams St',
 'State St & 35th St',
 'Halsted St & Wrightwood Ave',
 'St. Clair St & Erie St',
 'Dearborn Pkwy & Delaware Pl',
 'Dayton St & North Ave',
 'Larrabee St & North Ave',
 'Morgan Ave & 14th Pl',
 'Leavitt St & Archer Ave',
 'Sheridan Rd & Noyes St (NU)',
 'Dusable Harbor',
 'LaSalle St & Washington St',
 'Shields Ave & 31st St',
 'Aberdeen St & Monroe St',
 'Canal St & Monroe St (*)',
 'Ashland Ave & 21st St',
 'Clarendon Ave & Gordon Ter',
 'Franklin St & Lake St',
 'Ellis Ave & 60th St',
 'Wolcott Ave & Polk St',
 'Wells St & Elm St',
 'Ravenswood Ave & Lawrence Ave',
 'Halsted St & Diversey Pkwy',
 'Kedzie Ave & Milwaukee Ave',
 'Woodlawn Ave & 55th St',
 'Southport Ave & Irving Park Rd',
 'Western Ave & 21st St',
 'Wallace St & 35th St',
 'Franklin St & Jackson Blvd',
 'Racine Ave & Wrightwood Ave',
 'Ravenswood Ave & Irving Park Rd',
 'Broadway & Wilson Ave',
 'Ogden Ave & Roosevelt Rd',
 'Hampden Ct & Diversey Pkwy',
 'Paulina St & 18th St',
 'Wood St & Milwaukee Ave',
 'Ashland Ave & Lake St',
 'Clark St & Bryn Mawr Ave',
 'Clark St & Grace St',
 'Damen Ave & Chicago Ave',
 'Clybourn Ave & Division St',
 'Wood St & Taylor St',
 'Lake Shore Dr & North Blvd',
 'Broadway & Granville Ave',
 'Franklin St & Quincy St',
 'Ogden Ave & Chicago Ave',
 'Franklin St & Monroe St',
 'Lake Shore Dr & Ohio St',
 'Milwaukee Ave & Grand Ave',
 'LaSalle St & Adams St',
 'Clark St & Elmdale Ave',
 'Morgan St & Polk St',
 'Halsted St & North Branch St',
 'Halsted St & Archer Ave',
 'Damen Ave & Augusta Blvd',
 'Loomis St & Taylor St (*)',
 'Wells St & Polk St',
 'Larrabee St & Division St',
 'Indiana Ave & Roosevelt Rd',
 'Indiana Ave & 31st St',
 'Ashland Ave & Grace St',
 'Wood St & Hubbard St',
 'Wabash Ave & 16th St',
 'Southport Ave & Belmont Ave',
 'Racine Ave & Randolph St',
 'Clarendon Ave & Leland Ave',
 'Calumet Ave & 18th St',
 'Damen Ave & Grand Ave',
 'Sheffield Ave & Kingsbury St',
 'Desplaines St & Randolph St',
 'Lincoln Ave & Belle Plaine Ave',
 'Wabash Ave & Adams St',
 'Wentworth Ave & 35th St',
 'Lincoln Ave & Winona St',
 'Larrabee St & Oak St',
 'Adler Planetarium',
 'Benson Ave & Church St',
 'Marshfield Ave & Cortland St',
 'Stave St & Armitage Ave',
 'Bissell St & Armitage Ave',
 'Lincoln Ave & Leavitt St',
 'Desplaines St & Jackson Blvd',
 'Canal St & Taylor St',
 'Racine Ave & Fullerton Ave',
 'Burnham Harbor',
 'Wabash Ave & Cermak Rd',
 'Morgan St & 31st St',
 'Ravenswood Ave & Berteau Ave',
 'Jefferson St & Monroe St',
 'LaSalle St & Jackson Blvd',
 'Ogden Ave & Race Ave',
 'Clifton Ave & Lawrence Ave',
 'Ritchie Ct & Banks St',
 'Noble St & Milwaukee Ave',
 'Marine Dr & Ainslie St',
 'Fairbanks Ct & Grand Ave',
 'Western Ave & Division St',
 'Michigan Ave & Pearson St',
 'Racine Ave (May St) & Fulton St',
 'California Ave & Montrose Ave',
 'Field Blvd & South Water St',
 'Ashland Ave & Grand Ave',
 'Halsted St & Maxwell St',
 'Ashland Ave & Chicago Ave',
 'Damen Ave & Wellington Ave',
 'Southport Ave & Clybourn Ave',
 'Ashland Ave & Belle Plaine Ave',
 'Stetson Ave & South Water St',
 'Shore Dr & 55th St',
 'Western Ave & Congress Pkwy',
 'Wentworth Ave & Archer Ave',
 'Ridge Blvd & Touhy Ave',
 'Sheridan Rd & Loyola Ave',
 'Monticello Ave & Irving Park Rd',
 'Clinton St & Roosevelt Rd',
 'Sangamon St & Washington Blvd (*)',
 'Sheridan Rd & Buena Ave',
 'Halsted St & 35th St (*)',
 'Manor Ave & Leland Ave',
 'Knox Ave & Montrose Ave',
 'Lincoln Ave & Diversey Pkwy',
 'Broadway & Belmont Ave',
 'Larrabee St & Menomonee St',
 'Field Museum',
 'Theater on the Lake',
 'Halsted St & 37th St',
 'Central St Metra',
 'Loomis St & Jackson Blvd',
 'Michigan Ave & Madison St',
 'Loomis St & Archer Ave',
 'Wacker Dr & Washington St',
 'Hermitage Ave & Polk St',
 'Saginaw Ave & Exchange Ave',
 'Kedzie Ave & Foster Ave',
 'Ashland Ave & Archer Ave',
 'Prairie Ave & Garfield Blvd',
 'Kimbark Ave & 53rd St',
 'Western Ave & Winnebago Ave',
 'Rockwell St & Eastwood Ave',
 'Leavitt St & Armitage Ave',
 'Kedzie Ave & Leland Ave',
 'South Shore Dr & 74th St',
 'Clark St & Schiller St',
 'Orleans St & Chestnut St (NEXT Apts)',
 'Ashland Ave & McDowell Ave',
 'Lake Park Ave & 56th St',
 'Clinton St & Tilden St',
 'Oak Park Ave & Harrison St',
 'Racine Ave & Congress Pkwy',
 'Lake Park Ave & 35th St',
 'Halsted St & Roosevelt Rd',
 'Kimball Ave & Belmont Ave',
 'Clark St & Lunt Ave',
 'Lincoln Ave & Waveland Ave',
 'Eastlake Ter & Rogers Ave',
 'Elston Ave & Wabansia Ave',
 'Keystone Ave & Fullerton Ave',
 'Avondale Ave & Irving Park Rd',
 'Marion St & South Blvd',
 'California Ave & Altgeld St',
 'Orleans St & Elm St (*)',
 'Canal St & Harrison St',
 'Lake Shore Dr & Diversey Pkwy',
 'Lombard Ave & Garfield St',
 'Halsted St & 18th St',
 'Damen Ave & Melrose Ave',
 'State St & 19th St',
 'Shedd Aquarium',
 'Oakley Ave & Touhy Ave',
 'Spaulding Ave & Armitage Ave',
 'Oakley Ave & Roscoe St',
 'Drake Ave & Fullerton Ave',
 'Ashland Ave & Augusta Blvd',
 'Michigan Ave & 14th St',
 'Chicago Ave & Washington St',
 'Sheridan Rd & Greenleaf Ave',
 'Clark St & Armitage Ave',
 'Michigan Ave & 18th St',
 'Dodge Ave & Church St',
 'Ravenswood Ave & Montrose Ave (*)',
 'Lincoln Ave & Addison St',
 '900 W Harrison St',
 'Museum of Science and Industry',
 'Lakefront Trail & Bryn Mawr Ave',
 'Paulina St & Montrose Ave',
 'Francisco Ave & Foster Ave',
 'Damen Ave & Foster Ave',
 'Lake Shore Dr & Wellington Ave',
 'DuSable Museum',
 'Oakley Ave & Irving Park Rd',
 'East Ave & Garfield St',
 'Clark St & Leland Ave',
 'Damen Ave & Charleston St',
 'Drake Ave & Addison St',
 'Cottage Grove Ave & 47th St',
 'Clarendon Ave & Junior Ter',
 'Southport Ave & Wellington Ave',
 'St. Louis Ave & Balmoral Ave',
 'Larrabee St & Armitage Ave',
 'Damen Ave & Clybourn Ave',
 'Ogden Ave & Congress Pkwy',
 'Clinton St & 18th St',
 'Milwaukee Ave & Rockwell St',
 'Cannon Dr & Fullerton Ave',
 'Damen Ave & Cullerton St',
 'Montrose Harbor',
 'May St & Cullerton St',
 'Cornell Ave & Hyde Park Blvd',
 'California Ave & Byron St',
 'Lake Park Ave & 53rd St',
 'MLK Jr Dr & 47th St',
 'Western Ave & Leland Ave',
 'Ashland Ave & Wrightwood Ave',
 'Dorchester Ave & 63rd St',
 'Clark St & Schreiber Ave',
 'Blackstone Ave & Hyde Park Blvd',
 'Central Park Ave & Elbridge Ave',
 'Pulaski Rd & Lake St',
 'Clark St & 9th St (AMLI)',
 'Morgan St & 18th St',
 'Ravenswood Ave & Balmoral Ave',
 'Austin Blvd & Lake St',
 'Calumet Ave & 21st St',
 'Racine Ave & 13th St',
 'Lombard Ave & Madison St',
 'Oak Park Ave & South Blvd',
 'Budlong Woods Library',
 'Forest Ave & Lake St',
 'Woodlawn Ave & Lake Park Ave',
 'Western Ave & Monroe St',
 '2112 W Peterson Ave',
 'State St & 29th St',
 'Indiana Ave & 26th St',
 'Avers Ave & Belmont Ave',
 'Ashland Ave & 13th St',
 'Fairfield Ave & Roosevelt Rd',
 'California Ave & 21st St',
 'Washtenaw Ave & Lawrence Ave',
 'Seeley Ave & Roscoe St',
 'Racine Ave & 15th St',
 'Talman Ave & Addison St',
 'Lake Park Ave & 47th St',
 'Wentworth Ave & 24th St',
 'Emerald Ave & 28th St',
 'Claremont Ave & Hirsch St',
 'Cottage Grove Ave & 43rd St',
 'Kosciuszko Park',
 'Princeton Ave & Garfield Blvd',
 'Greenwood Ave & 47th St',
 'Albany Ave & Bloomingdale Ave',
 'East Ave & Madison St',
 'Normal Ave & Archer Ave',
 'Kilbourn Ave & Irving Park Rd',
 'Broadway & Thorndale Ave',
 'Damen Ave & Sunnyside Ave',
 'MLK Jr Dr & Oakwood Blvd',
 'Wentworth Ave & 33rd St',
 'Washtenaw Ave & 15th St (*)',
 'Troy St & North Ave',
 'Campbell Ave & North Ave',
 'Clark St & Touhy Ave',
 'Forest Ave & Chicago Ave',
 'Elmwood Ave & Austin St',
 'Cuyler Ave & Augusta St',
 'Humboldt Blvd & Armitage Ave',
 'Emerald Ave & 31st St',
 'Jeffery Blvd & 76th St',
 'California Ave & Francis Pl',
 'Austin Blvd & Chicago Ave',
 'Leavitt St & Addison St',
 'Wisconsin Ave & Madison St',
 'Spaulding Ave & Division St',
 'Bosworth Ave & Howard St',
 'Princeton Ave & 47th St',
 'Albany Ave & 26th St',
 'Ridgeland Ave & Lake St',
 'Troy St & Elston Ave',
 'Clinton St & Polk St (*)',
 'Valli Produce - Evanston Plaza',
 'Fort Dearborn Dr & 31st St',
 'Kedzie Ave & Lake St',
 'Maplewood Ave & Peterson Ave',
 'Kedzie Ave & Chicago Ave',
 'Halsted St & 21st St',
 'Stony Island Ave & 82nd St',
 'Yates Blvd & 75th St',
 'Stony Island Ave & 67th St',
 'Wells St & 19th St',
 'Cottage Grove Ave & Oakwood Blvd',
 'Kedzie Ave & 21st St',
 'Shields Ave & 28th Pl',
 'Prairie Ave & 43rd St',
 'Clark St & Jarvis Ave',
 'California Ave & 23rd Pl',
 'Calumet Ave & 35th St',
 'Warren Park East',
 'Dorchester Ave & 49th St',
 'Damen Ave & Pershing Rd',
 'Racine Ave & 35th St',
 'California Ave & 26th St',
 'Cottage Grove Ave & 63rd St',
 'Warren Park West',
 'Sheridan Rd & Lawrence Ave',
 'MLK Jr Dr & 63rd St',
 'Central St & Girard Ave',
 'Albany (Kedzie) Ave & Montrose Ave',
 'Western Ave & 24th St',
 'California Ave & Fletcher St',
 'Eberhart Ave & 61st St',
 'Western Ave & Howard St',
 'Ridge Blvd & Howard St',
 'Millard Ave & 26th St',
 'Sacramento Blvd & Franklin Blvd',
 'California Ave & Lake St',
 'Wolcott Ave & Fargo Ave',
 'Elizabeth St & 47th St',
 'Laramie Ave & Gladys Ave',
 'Hoyne Ave & 47th St',
 'Western Ave & 28th St',
 'Halsted St & 63rd St',
 'South Shore Dr & 71st St',
 'Kedzie Ave & Roosevelt Rd',
 'Pulaski Rd & Eddy St',
 'Humphrey Ave & Ontario St',
 'Sawyer Ave & Irving Park Rd',
 'Glenwood Ave & Touhy Ave',
 'Central Park Ave & North Ave',
 'Kilbourn Ave & Milwaukee Ave',
 'Western Blvd & 48th Pl',
 'Pulaski Rd & Congress Pkwy',
 'Stony Island Ave & 71st St',
 'MLK Jr Dr & 56th St (*)',
 'State St & Pershing Rd',
 'Western Ave & Lunt Ave',
 'Central Ave & Harrison St',
 'Union Ave & Root St',
 'Halsted St & 47th Pl',
 'Conservatory Dr & Lake St',
 'Phillips Ave & 79th St',
 'Wood St & 35th St',
 'South Shore Dr & 67th St',
 'Morgan St & Pershing Rd',
 'Ashland Ave & Pershing Rd',
 'Jeffery Blvd & 71st St',
 'Kedzie Ave & 24th St',
 'Damen Ave & Coulter St',
 'Stony Island Ave & 64th St',
 'MLK Jr Dr & 83rd St',
 'Keystone Ave & Montrose Ave',
 'Central Park Ave & Ogden Ave',
 'Jeffery Blvd & 67th St',
 '63rd St Beach',
 'Cottage Grove Ave & 51st St',
 'Central Ave & Lake St',
 'Wabash Ave & 87th St',
 'Rainbow Beach',
 'Ashland Ave & 50th St',
 'Kedzie Ave & Harrison St',
 'Kedzie Ave & Bryn Mawr Ave',
 'Evans Ave & 75th St',
 'Cicero Ave & Lake St',
 'Cottage Grove Ave & 83rd St',
 'Damen Ave & 51st St',
 'Stony Island Ave & 75th St',
 'Milwaukee Ave & Cuyler Ave',
 'May St & 69th St',
 'Wentworth Ave & 63rd St',
 'Central Ave & Chicago Ave',
 'Halsted St & 69th St',
 'Cottage Grove Ave & 67th St',
 'Mason Ave & Madison St',
 'Marshfield Ave & 44th St',
 'Cottage Grove Ave & 71st St',
 'Central Park Blvd & 5th Ave',
 'Commercial Ave & 83rd St',
 'Halsted St & 56th St',
 'Racine Ave & 61st St',
 'Cottage Grove Ave & 78th St',
 'Halsted St & 51st St',
 'Ashland Ave & 63rd St',
 'Vernon Ave & 75th St',
 'Pulaski Rd & Madison St',
 'Kostner Ave & Lake St',
 'Greenwood Ave & 79th St',
 'Central Ave & Madison St',
 'Cicero Ave & Flournoy St',
 'Seeley Ave & Garfield Blvd',
 'Woodlawn Ave & 75th St',
 'Normal Ave & 72nd St',
 'Halsted St & 59th St',
 'Cicero Ave & Quincy St',
 'Ashland Ave & 69th St',
 'Ashland Ave & 66th St',
 'Throop St & 52nd St',
 'Kostner Ave & Adams St',
 'Bennett Ave & 79th St',
 'Laramie Ave & Madison St',
 'Stony Island Ave & South Chicago Ave',
 'Perry Ave & 69th St',
 'California Ave & Cortez St',
 'Michigan Ave & 8th St',
 'Damen Ave & 59th St',
 'Marshfield Ave & 59th St',
 'State St & 76th St',
 'Calumet Ave & 71st St',
 'Wabash Ave & 83rd St',
 'Exchange Ave & 79th St',
 'Vernon Ave & 79th St',
 'Shields Ave & 43rd St',
 'Laramie Ave & Kinzie St',
 'Ellis Ave & 83rd St',
 'South Chicago Ave & 83rd St',
 'Phillips Ave & 82nd St',
 'Racine Ave & Garfield Blvd',
 'Ashland Ave & Garfield Blvd',
 'Racine Ave & 65th St',
 'LaSalle (Wells) St & Huron St',
 'Wabash Ave & 9th St',
 'Honore St & Division St',
 'Eberhart (Vernon) Ave & 79th St',
 'Albany Ave & Montrose Ave',
 'Dearborn St & Van Buren St (*)',
 'Chicago Ave & Dempster St',
 'Canal St & Jackson Blvd',
 'Washtenaw Ave & Ogden Ave (*)',
 'Austin Blvd & Madison St',
 'Delano Ct & Roosevelt Rd']

In [10]:
len(allSt17)

593

In [11]:
removed17 = list(set(allSt17) - set(allSt18))
len(removed17)

27

In [12]:
added18 = list(set(allSt18) - set(allSt17))
len(added18)

56

# Plotting all added stations of 2018 and removed stations of 2017

In [13]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [14]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [15]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [16]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [17]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [18]:
#Adding co-ordinates to all the stations that are added this year
n18 = {k: locations[k] for k in locations.keys() & set(added18)}

#printing out the first item in the dictionary to check the key-value pair
list(n18.items())[0]

('Clark St & Newport St', {'lat': 41.94454, 'long': -87.654678})

In [19]:
#Converting the dictionary into dataframe
st18 = pd.DataFrame.from_dict(n18, orient='index')

In [20]:
#Resetting the index
st18.reset_index(inplace=True)

#Renaming the index column to Station name
st18 = st18.rename(columns = {'index':'Station Name'})

In [21]:
#Adding a column with values of color for every station
st18['color'] = st18.apply(lambda row: 'white', axis=1)

#printing to check the structure of the dataframe
st18.head(1)

,Station Name,lat,long,color
0,Clark St & Newport St,41.94454,-87.654678,white


In [22]:
removed17 = ['Damen Ave & Pierce Ave',
 'LaSalle (Wells) St & Huron St',
 'Ravenswood Ave & Montrose Ave (*)',
 'Clark St & Winnemac Ave',
 'Halsted St & Diversey Pkwy',
 'Financial Pl & Congress Pkwy',
 'Oakley Ave & Roscoe St',
 'Damen Ave & Augusta Blvd',
 'Ashland Ave & Lake St',
 'Washtenaw Ave & 15th St (*)',
 'Halsted St & Blackhawk St (*)',
 'Wisconsin Ave & Madison St',
 'Pulaski Rd & Eddy St',
 'Wentworth Ave & Archer Ave',
 'Orleans St & Ohio St',
 'Wood St & Division St',
 'Phillips Ave & 82nd St',
 'Clifton Ave & Lawrence Ave',
 'Ravenswood Ave & Balmoral Ave',
 'MLK Jr Dr & Oakwood Blvd',
 'Wabash Ave & 8th St',
 'Vernon Ave & 79th St',
 'Mason Ave & Madison St',
 'Clinton St & Jackson Blvd',
 'Pulaski Rd & Madison St',
 'Lawndale Ave & 23rd St',
 'Albany (Kedzie) Ave & Montrose Ave',
 'Hampden Ct & Diversey Pkwy']

In [23]:
#adding co-ordinates for the removed stations in 2017
rm17 = {k: locations[k] for k in locations.keys() & set(removed17)}

#printing out the first item in the dictionary to check the key-value pair
list(rm17.items())[0]

('Phillips Ave & 82nd St', {'lat': 41.746334, 'long': -87.564725})

In [24]:
#Converting the dictionary into dataframe
rm17 = pd.DataFrame.from_dict(rm17, orient='index')

In [25]:
#Resetting the index
rm17.reset_index(inplace=True)

#Renaming the index column to Station name
rm17 = rm17.rename(columns = {'index':'Station Name'})

In [26]:
#Adding a column with values of color for every station
rm17['color'] = rm17.apply(lambda row: 'pink', axis=1)

#printing to check the structure of the dataframe
rm17.head(1)

,Station Name,lat,long,color
0,Phillips Ave & 82nd St,41.746334,-87.564725,pink


# Location Marker map for interactivity purpose (Station Name)

In [27]:
#merging both the added18 and removed17 dataframes
merged = pd.concat([rm17, st18], axis =0)

In [28]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2018
## Saving the map for converting into gif

In [29]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2018')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "green",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st18.apply(plotDot, axis = 1)

m

In [30]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [31]:
m.save('Gif_circlemarker/2018.html')

In [32]:
with fileinput.FileInput('Gif_circlemarker/2018.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [33]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2018.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2018.png')
browser.quit()

In [34]:
image = Image.open('ss2018.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2018.png')

# Heatmap 2018
## Saving the map for converting into gif

In [35]:
numlist = st18.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2018')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [36]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [37]:
m.save('Gif_heatmap/2018.html')

In [38]:
with fileinput.FileInput('Gif_heatmap/2018.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [39]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2018.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2018.png')
browser.quit()

In [41]:
image = Image.open('ss2018.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2018.png')